In [1]:
import math
import argparse
import pandas as pd
import random
import csv
import time

import matplotlib.pyplot as plt
import torch
import numpy as np
from scipy.stats import boxcox

from mhcflurry import Class1AffinityPredictor

from torchvision import transforms, utils
from torch import nn 
import torch.optim as optim
import torch.backends.cudnn as cudnn
import torch.nn.functional as F

import metrics
import data_transformation
import train_validation
import model_architecture
import rank_model
import utils
import unittest
from sklearn.model_selection import train_test_split  
from sklearn.preprocessing import StandardScaler

In [2]:
def test_modules(module_test):
    """Run unittest in imported files"""
    
    suite = unittest.TestLoader().loadTestsFromModule(module_test)
    unittest.TextTestRunner(verbosity=1).run(suite)

In [3]:
testmodules = [data_transformation.data_class_test(), metrics.metrics_test(), utils.utils_test()]
for test in testmodules:
    test_modules(test)

..
----------------------------------------------------------------------
Ran 2 tests in 17.098s

OK
..
----------------------------------------------------------------------
Ran 2 tests in 0.005s

OK
..
----------------------------------------------------------------------
Ran 2 tests in 0.102s

OK


In [4]:
params = {
    'batch_size': 64,
    'save_model_path': "ResNet_small.chkpt",
    'epoch': 200,
    'model': "ResNet_small",
    'dropout': 0.2,
    "optimizer": "sgd", #adam sgd
    "learning_rate": 0.001,
    "adam_vanilla": True,
    'momentum': 0.9,
    'preproc': "log", #boxcox log
    'weight_decay': 0
}

In [5]:
#data_class = data_transformation.data_transformation(path_data = "/data/curated_training_data_no_mass_spec.csv",
#                                                     path_mhc = "/data/aligned_mhc_dataset.csv",
#                                                     allele_name = None,
#                                                     quant_data = True,
#                                                     encoding = "one-hot")
#
#pep, mhc, target = data_class.__getitem__()
#
#pep = np.expand_dims(pep, axis=1)
#mhc = np.expand_dims(mhc, axis=1)
#inp = np.hstack((pep, mhc))
#print(inp.shape)
#
#X_train, X_test, y_train, y_test = train_test_split(inp, target.T, test_size=0.2, random_state=42)
data_class = data_transformation.data_transformation(path_data = "/data/data_curated_20180219/curated_training_data_no_mass_spec_dbscan.csv",
                                                     path_mhc = "/data/aligned_mhc_dataset.csv",
                                                     allele_name = None,
                                                     quant_data = True,
                                                     encoding = "one-hot",
                                                     transformation = params['preproc'])


pep, mhc, target, dbscan = data_class.__getitem__()

if params['preproc'] == 'boxcox':
    target = boxcox(target.flatten(), 0.055).reshape(-1, 1)
else:
    target = target.T
pep = np.expand_dims(pep, axis=1)
mhc = np.expand_dims(mhc, axis=1)
inp = np.hstack((pep, mhc))
print(inp.shape)

np.random.seed(17) 
test_index = np.random.choice(np.where(dbscan.T == -1)[0],
                              size = int(np.round(dbscan.T.shape[0] * 0.2)),
                              replace = False)

X_train = inp[~np.isin(np.arange(len(inp)), test_index)]
X_test = inp[np.isin(np.arange(len(inp)), test_index)]
y_train = target[~np.isin(np.arange(len(target)), test_index)]
y_test = target[np.isin(np.arange(len(target)), test_index)]

if params['preproc'] == 'boxcox':
    scaler = StandardScaler()
    scaler.fit(y_train)
    y_train = scaler.transform(y_train)
    y_test = scaler.transform(y_test)

if params['model'] == 'conv3x3':
    model = model_architecture.conv3x3(inputchannel = np.size(X_train, 3),
                                    L = np.size(X_train, 2),
                                    dropout = params['dropout'],
                                    dropoutearly = 0)#0.2)
elif params['model'] == 'convnet':
    model = model_architecture.convnet(inputchannel = np.size(X_train, 3),
                                L = np.size(X_train, 2),
                                dropout = params['dropout'])
elif params['model'] == 'ResNet18':
    model = model_architecture.ResNet(inputchannel = np.size(X_train, 3),
                                    block = model_architecture.BasicBlock, 
                                    num_blocks = [2,2,2,2],
                                    num_classes = 1,
                                    dropout = params['dropout'])
elif params['model'] == 'ResNet50':
    model = model_architecture.ResNet(inputchannel = np.size(X_train, 3),
                                    block = model_architecture.Bottleneck, 
                                    num_blocks = [3,4,6,3],
                                    num_classes = 1,
                                    dropout = params['dropout'])
elif params['model'] == 'ResNet_small':
    model = model_architecture.ResNet(inputchannel = np.size(X_train, 3),
                                    block = model_architecture.BasicBlock, 
                                    num_blocks = [1,1,1,1],
                                    num_classes = 1,
                                    dropout = params['dropout'])
  
pytorch_total_params = sum(p.numel() for p in model.parameters())
print("Model parameters: " + str(pytorch_total_params) )
if torch.cuda.device_count() > 0:
    print('Using GPU' + str(utils.pick_gpu_lowest_memory()))
    device = torch.device('cuda:' + str(utils.pick_gpu_lowest_memory()))
else:
    print('Using CPU')
    device = torch.device('cpu')
    
model = model.to(device)
criterion = metrics.select_criterion('MSE')
if params['optimizer'] == "adam":
    if params['adam_vanilla']:
        optimizer = optim.Adam(
                      filter(lambda x: x.requires_grad, model.parameters()),
                      lr = params['learning_rate'],
                      weight_decay = params['weight_decay'])
    else:
        optimizer = optim.Adam(
                      filter(lambda x: x.requires_grad, model.parameters()),
                      lr = params['learning_rate'],
                      betas=(0.9, 0.98),
                      eps=1e-09,
                      weight_decay = params['weight_decay'])    
elif params['optimizer'] == "sgd":
    optimizer = optim.SGD(
                filter(lambda x: x.requires_grad, model.parameters()),
                lr=params['learning_rate'], 
                momentum=params['momentum'],
                weight_decay = params['weight_decay'])


train_data = torch.utils.data.TensorDataset(torch.from_numpy(X_train).float(), torch.from_numpy(y_train))
test_data = torch.utils.data.TensorDataset(torch.from_numpy(X_test).float(), torch.from_numpy(y_test))

train_dataloader = torch.utils.data.DataLoader(train_data,
                              batch_size = params['batch_size'],
                              shuffle = True, 
                              drop_last = True)
eval_dataloader = torch.utils.data.DataLoader(test_data,
                              batch_size = params['batch_size'],
                              shuffle = True,
                              drop_last = True)

(156996, 2, 34, 20)


NameError: name 'model' is not defined

In [ ]:
valid_losss, train_losses, valid_accus, train_accus, true_valid, pred_valid = train_validation.start_training(params['save_model_path'], params['epoch'], model,
                                                                                                              train_dataloader, eval_dataloader, optimizer,
                                                                                                              device, criterion)

In [ ]:
valid_losss, train_losses, valid_accus, train_accus, true_valid, pred_valid = train_validation.continue_training(params['save_model_path'], params['epoch'], model, 
                                                                                                                 train_dataloader, eval_dataloader, optimizer, 
                                                                                                                 device, criterion)

In [ ]:
csv_columns = ['id', 'Loss', 'Accuracy', 'Model', 'Dropout',
               'Optimizer', 'Adam_vanilla', 'momentum', 
               'Learning_rate', 'Preprocess', 'Batch_size',
               'Weight_decay']

dict_data = []
time_exp = round(time.time())
for i in range(int(params['epoch'] / 5)):
    dict_data.append({'id': str(time_exp) + '_' + str(i*5+1),
                      'Loss': (valid_losss[i*5] / (np.max(y_test) - np.min(y_test))),
                      'Accuracy': valid_accus[i*5], 'Model': params['model'], 'Dropout': params['dropout'],
                      'Optimizer': params['optimizer'], 'Adam_vanilla': params['adam_vanilla'],
                      'momentum': params['momentum'], 'Learning_rate': params['learning_rate'],
                      'Preprocess': params['preproc'], 'Batch_size': params['batch_size'],
                      'Weight_decay': params['weight_decay']})

csv_file = "experiment_log.csv"
if os.path.isfile(csv_file):
    fileEmpty = os.stat(csv_file).st_size == 0
else:
    fileEmpty = True
try:
    with open(csv_file, 'a') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames = csv_columns)
        if fileEmpty:
            writer.writeheader()  # file doesn't exist yet, write a header
        for data in dict_data:
            writer.writerow(data)
except IOError:
    print("I/O error")

## Predict new Values

In [5]:
def predict_mhc(allele_name):

    #data_class = data_transformation.data_transformation(path_data = "/data/curated_training_data_no_mass_spec.csv",
    #                                                     path_mhc = "/data/aligned_mhc_dataset.csv",
    #                                                     allele_name = None,
    #                                                     quant_data = True,
    #                                                     encoding = "one-hot")
    #
    #pep, mhc, target = data_class.__getitem__()
    #
    #pep = np.expand_dims(pep, axis=1)
    #mhc = np.expand_dims(mhc, axis=1)
    #inp = np.hstack((pep, mhc))
    #print(inp.shape)
    #
    #X_train, X_test, y_train, y_test = train_test_split(inp, target.T, test_size=0.2, random_state=42)
    data_class = data_transformation.data_transformation(path_data = "/data/test_data_all_allele.csv",
                                                         path_mhc = "/data/aligned_mhc_dataset.csv",
                                                         allele_name = allele_name,
                                                         quant_data = True,
                                                         dbscan = False,
                                                         encoding = "one-hot",
                                                         transformation = params['preproc'])


    pep, mhc, target = data_class.__getitem__()

    if params['preproc'] == 'boxcox':
        target = boxcox(target.flatten(), 0.055).reshape(-1, 1)
    else:
        target = target.T
    pep = np.expand_dims(pep, axis=1)
    mhc = np.expand_dims(mhc, axis=1)
    inp = np.hstack((pep, mhc))
    print(inp.shape)

    if params['preproc'] == 'boxcox':
        scaler = StandardScaler()
        scaler.fit(y_train)
        y_train = scaler.transform(y_train)
        y_test = scaler.transform(y_test)

    if params['model'] == 'conv3x3':
        model = model_architecture.conv3x3(inputchannel = np.size(inp, 3),
                                        L = np.size(inp, 2),
                                        dropout = params['dropout'],
                                        dropoutearly = 0)#0.2)
    elif params['model'] == 'convnet':
        model = model_architecture.convnet(inputchannel = np.size(inp, 3),
                                    L = np.size(inp, 2),
                                    dropout = params['dropout'])
    elif params['model'] == 'ResNet18':
        model = model_architecture.ResNet(inputchannel = np.size(inp, 3),
                                        block = model_architecture.BasicBlock, 
                                        num_blocks = [2,2,2,2],
                                        num_classes = 1,
                                        dropout = params['dropout'])
    elif params['model'] == 'ResNet50':
        model = model_architecture.ResNet(inputchannel = np.size(inp, 3),
                                        block = model_architecture.Bottleneck, 
                                        num_blocks = [3,4,6,3],
                                        num_classes = 1,
                                        dropout = params['dropout'])
    elif params['model'] == 'ResNet_small':
        model = model_architecture.ResNet(inputchannel = np.size(inp, 3),
                                        block = model_architecture.BasicBlock, 
                                        num_blocks = [1,1,1,1],
                                        num_classes = 1,
                                        dropout = params['dropout'])

    pytorch_total_params = sum(p.numel() for p in model.parameters())
    print("Model parameters: " + str(pytorch_total_params) )
    if torch.cuda.device_count() > 0:
        print('Using GPU' + str(utils.pick_gpu_lowest_memory()))
        device = torch.device('cuda:' + str(utils.pick_gpu_lowest_memory()))
    else:
        print('Using CPU')
        device = torch.device('cpu')

    model = model.to(device)
    criterion = metrics.select_criterion('MSE')
    if params['optimizer'] == "adam":
        if params['adam_vanilla']:
            optimizer = optim.Adam(
                          filter(lambda x: x.requires_grad, model.parameters()),
                          lr = params['learning_rate'],
                          weight_decay = params['weight_decay'])
        else:
            optimizer = optim.Adam(
                          filter(lambda x: x.requires_grad, model.parameters()),
                          lr = params['learning_rate'],
                          betas=(0.9, 0.98),
                          eps=1e-09,
                          weight_decay = params['weight_decay'])    
    elif params['optimizer'] == "sgd":
        optimizer = optim.SGD(
                    filter(lambda x: x.requires_grad, model.parameters()),
                    lr=params['learning_rate'], 
                    momentum=params['momentum'],
                    weight_decay = params['weight_decay'])


    test_data = torch.utils.data.TensorDataset(torch.from_numpy(inp).float(), torch.from_numpy(target))

    test_dataloader = torch.utils.data.DataLoader(test_data,
                              batch_size = 1,
                              shuffle = False,
                              drop_last = False)

    a, b = train_validation.prediction_only(params['save_model_path'], model, test_dataloader, optimizer, device, criterion)
    out_pred = np.concatenate(a).ravel()
    out_true = np.concatenate(b).ravel()
    mymodel_out = pd.DataFrame(list(zip(out_pred, out_true)), columns =['pred', 'true']).apply(lambda y: rank_model.reverse_log_transformation(y))

    flurry_data = pd.read_csv("C:/Users/paul_/OneDrive/Desktop/master-thesis/data/test_data_all_allele.csv")
    mask = (flurry_data['peptide'].str.len() >= 8) & (flurry_data['peptide'].str.len() <= 15)
    flurry_data = flurry_data.loc[mask]
    flurry_out = rank_model.mhcflurry_test(flurry_data, allele_name)
    
    error_dict_my, len_dict_my = rank_model.root_mean_squared(mymodel_out, allele_name, "my")
    error_dict_flurry, len_dict_flurry = rank_model.root_mean_squared(flurry_out, allele_name, "flurry")
    return(error_dict_my, len_dict_my, error_dict_flurry, len_dict_flurry)

In [20]:
# More MHCs are supported in my model
# No restriction for length of peptides -- mhcflurry 8 - 15 len

params['model'] = "ResNet_small"
params['save_model_path'] = "ResNet_small025dropout.chkpt"

# Compare only MHC supported in mhcflurry
allele_names = pd.read_csv("C:/Users/paul_/OneDrive/Desktop/master-thesis/data/test_data_all_allele.csv").allele.unique()
predictor = Class1AffinityPredictor.load()

mhc_names = list(set(allele_names.tolist()) & set(predictor.supported_alleles))
combined_dict = {}
len_dict = {}
for i in mhc_names:
    print(i)
    error_my, len_my, error_flurry, len_flurry = predict_mhc(i)
    len_dict.update(len_my)
    len_dict.update(len_flurry)
    combined_dict.update(error_my)
    combined_dict.update(error_flurry)

HLA-A*03:01
(10, 2, 34, 20)
Model parameters: 4919937
Using CPU
Checkpoint found and loaded - Predict values
[ Epoch 1 ]


  - (Test) Elapse: 0.005728610356648763 min
HLA-A*24:02
(8, 2, 34, 20)
Model parameters: 4919937
Using CPU
Checkpoint found and loaded - Predict values
[ Epoch 1 ]


  - (Test) Elapse: 0.005364545186360677 min
HLA-A*68:01
(2, 2, 34, 20)
Model parameters: 4919937
Using CPU
Checkpoint found and loaded - Predict values
[ Epoch 1 ]


  - (Test) Elapse: 0.0018236080805460611 min
HLA-A*02:05
(2, 2, 34, 20)
Model parameters: 4919937
Using CPU
Checkpoint found and loaded - Predict values
[ Epoch 1 ]


  - (Test) Elapse: 0.0015625874201456705 min
H-2-Kb
(49, 2, 34, 20)
Model parameters: 4919937
Using CPU
Checkpoint found and loaded - Predict values
[ Epoch 1 ]


...

  - (Test) Elapse: 0.0014131704966227213 min
HLA-B*44:03
(4, 2, 34, 20)
Model parameters: 4919937
Using CPU
Checkpoint found and loaded - Predict values
[ Epoch 1 ]


  - (Test) Elapse: 0.002997310956319173 min
H-2-Db
(62, 2, 34, 20)
Model parameters: 4919937
Using CPU
Checkpoint found and loaded - Predict values
[ Epoch 1 ]


  - (Test) Elapse: 0.035086750984191895 min
HLA-B*15:01
(5, 2, 34, 20)
Model parameters: 4919937
Using CPU
Checkpoint found and loaded - Predict values
[ Epoch 1 ]


  - (Test) Elapse: 0.0040855805079142255 min
HLA-B*38:01
(2, 2, 34, 20)
Model parameters: 4919937
Using CPU
Checkpoint found and loaded - Predict values
[ Epoch 1 ]


  - (Test) Elapse: 0.0014952460924784342 min


In [21]:
combined_dict

{'HLA-A*03:01my': 6628.023363915202,
 'HLA-A*03:01flurry': 467.068469477591,
 'HLA-A*24:02my': 10355.625933065807,
 'HLA-A*24:02flurry': 5708.133603554634,
 'HLA-A*68:01my': 29.87179134993718,
 'HLA-A*68:01flurry': 17.575974910700896,
 'HLA-A*02:05my': 26.069356869003535,
 'HLA-A*02:05flurry': 148.6809021300337,
 'H-2-Kbmy': 3611.8441815086994,
 'H-2-Kbflurry': 5670.493803096738,
 'HLA-B*51:01my': 1190.9107443561595,
 'HLA-B*51:01flurry': 260.3452569976687,
 'HLA-A*11:01my': 7274.252153930705,
 'HLA-A*11:01flurry': 7630.797311378845,
 'HLA-A*02:01my': 26141.516940120844,
 'HLA-A*02:01flurry': 26411.98437729448,
 'HLA-A*68:02my': 29.510797043027925,
 'HLA-A*68:02flurry': 24.83060501704868,
 'HLA-B*44:02my': 453.94168812789644,
 'HLA-B*44:02flurry': 181.76560422029905,
 'HLA-A*01:01my': 9471.208705460185,
 'HLA-A*01:01flurry': 10153.670854486432,
 'HLA-B*07:02my': 8169.236512904359,
 'HLA-B*07:02flurry': 9349.510131293062,
 'HLA-B*35:01my': 25574.300510184028,
 'HLA-B*35:01flurry': 27621

In [22]:
len_dict

{'HLA-A*03:01my': 10,
 'HLA-A*03:01flurry': 10,
 'HLA-A*24:02my': 8,
 'HLA-A*24:02flurry': 8,
 'HLA-A*68:01my': 2,
 'HLA-A*68:01flurry': 2,
 'HLA-A*02:05my': 2,
 'HLA-A*02:05flurry': 2,
 'H-2-Kbmy': 49,
 'H-2-Kbflurry': 48,
 'HLA-B*51:01my': 1,
 'HLA-B*51:01flurry': 1,
 'HLA-A*11:01my': 8,
 'HLA-A*11:01flurry': 8,
 'HLA-A*02:01my': 63,
 'HLA-A*02:01flurry': 63,
 'HLA-A*68:02my': 2,
 'HLA-A*68:02flurry': 2,
 'HLA-B*44:02my': 4,
 'HLA-B*44:02flurry': 4,
 'HLA-A*01:01my': 11,
 'HLA-A*01:01flurry': 11,
 'HLA-B*07:02my': 12,
 'HLA-B*07:02flurry': 12,
 'HLA-B*35:01my': 1,
 'HLA-B*35:01flurry': 1,
 'HLA-B*44:03my': 4,
 'HLA-B*44:03flurry': 4,
 'H-2-Dbmy': 62,
 'H-2-Dbflurry': 61,
 'HLA-B*15:01my': 5,
 'HLA-B*15:01flurry': 5,
 'HLA-B*38:01my': 2,
 'HLA-B*38:01flurry': 2}

In [23]:
weighted_a = np.average(a = np.array([value for key, value in combined_dict.items() if 'my' in key.lower()]),
                        weights = np.array([value for key, value in len_dict.items() if 'my' in key.lower()]))
a = np.mean(a = np.array([value for key, value in combined_dict.items() if 'my' in key.lower()]))
print(weighted_a)
print(a)

12092.855879564228
8312.952506830115


In [24]:
weighted_b = np.average(a = np.array([value for key, value in combined_dict.items() if 'flurry' in key.lower()]),
                        weights = np.array([value for key, value in len_dict.items() if 'flurry' in key.lower()]))
b = np.mean([value for key, value in combined_dict.items() if 'flurry' in key.lower()])
print(weighted_b)
print(b)

12540.931610166197
7845.8952222258595


In [25]:
print((a/b - 1) * 100)
print((weighted_a/weighted_b - 1) * 100)

5.952887100520732
-3.572906260319142


In [33]:
# ResNet_small 200e 0.25 dropout
(a/b - 1) * 100

-0.8685909777161993

In [28]:
# ResNet_small 75e 0.25 dropout
(a/b - 1) * 100

-3.5294610188582154

In [22]:
# ResNet_small 60e 0.25 dropout
(a/b - 1) * 100

-5.355991318651521

In [17]:
# ResNet_small 40e 0.25 dropout
(a/b - 1) * 100

-4.645966507406185

In [12]:
# ConvNet 200e
(a/b - 1) * 100

12.516955714370948

In [15]:
# ConvNet 110e
(a/b - 1) * 100

5.914762463525536

In [10]:
# ConvNet 50e
(a/b - 1) * 100

9.46196891540847

In [34]:
# ResNet18 - altes Splitting
(a/b - 1) * 100

3.102533941685026

In [25]:
# ResNet_small 160 Epoch
(a/b - 1) * 100

-2.3314228440623697

In [20]:
# No fcking clue conv3x3???
(a/b - 1) * 100

-4.276630118220703